<a href="https://colab.research.google.com/github/ilyakaliya/frap_data_processing/blob/main/frap_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.optimize import least_squares
from collections.abc import Iterable
import os
import re

from scipy.constants import k as k_b
from scipy.constants import zero_Celsius

In [ ]:
def process_frap_images(directory_path, curves_number, scan_size_um,
                        show_curves=0, show_baseline=False,
                        frames_without_bleaching=2, crop=(0.3, 0.03)):
    """
    Processes microscopic images of diffusion kinetics of a bleached horizontal line (FRAP)

    Parameters
    ----------
    directory_path : str
        Path to directory with microscopic images
    curves_number : int
        Number of Gaussians to fit the intensity distribution profiles
    scan_size_um : float
        Scanning area size, μm
    show_curves : int, optional
        Number of first intensity distributions to show, 0 by default
    show_baseline : bool, optional
        If True, plotted baseline is shown, False by default
    frames_without_bleaching : int, optional
        Number of images taken before bleaching, 2 by default
    crop : iterable, optional
        Relative size edges to crop, by default 0.3 vertically, 0.03 horizontally

    Returns
    ----------
    list : list(s) of full width at half minimum intensity of bleached line
    """

    image_filenames = os.listdir(directory_path)
    number_of_images = len(image_filenames) - frames_without_bleaching
    extension = image_filenames[0].split('.')[-1]
    prefix_end_index = re.search(f'\d+.{extension}', image_filenames[0]).start()
    image_filename_prefix = directory_path + '/' + image_filenames[0][:prefix_end_index]
    index_length = len(image_filenames[0]) - prefix_end_index - len('.png')

    print(number_of_images)

    def filename_postfix(index):
        index = str(index)
        return '0' * (index_length - len(index)) + index + '.' + extension

    with Image.open(image_filename_prefix + filename_postfix(0)) as initial_image:
        initial_image = initial_image.convert('L')
        initial_data = np.array(initial_image).astype(float) / 255
    image_size_px = initial_data.shape[0]
    crop_x = int(crop[0] * image_size_px)
    crop_y = int(crop[1] * image_size_px)
    initial_data = initial_data[crop_x:-crop_x, crop_y:-crop_y]
    initial_data = np.mean(initial_data, 1)

    x = np.linspace(0, len(initial_data) * scan_size_um / image_size_px, len(initial_data))

    k, b = np.polyfit(x, initial_data, 1)
    baseline = k * x + b

    if show_baseline:
        plt.plot(x, initial_data)
        plt.plot(x, baseline)
        plt.xlabel('x, μm')
        plt.ylabel('Intensity')
        plt.show()

    w = []
    w1 = []
    start_image = frames_without_bleaching
    end_image = min(60, number_of_images - 2)
    for i in range(start_image, end_image):
        with Image.open(image_filename_prefix + filename_postfix(i + frames_without_bleaching)) as image:
            image = image.convert('L')
            data = np.array(image).astype(float) / 255
        data = data[crop_x:-crop_x, crop_y:-crop_y]
        data = np.mean(data, 1)
        data /= baseline

        def double_gaussian_distribution(x, y0, a, x0, w, b, x1, w1):
            return y0 + a * np.exp(-2 * ((x - x0) ** 2) / (w ** 2)) + b * np.exp(-2 * ((x - x1) ** 2) / (w1 ** 2))
        def gaussian_distribution(x, y0, a, x0, w):
            return y0 + a * np.exp(-2 * ((x - x0) ** 2) / (w ** 2))
        if curves_number == 2:
            guess_params = data[0], min(data) - data[0], x[np.argmin(data)], 0.2 * image_size_px, (min(data) + data[0])/50, x[np.argmin(data[:round(0.5*len(x))])], 0.4 * image_size_px
            params, cov = curve_fit(double_gaussian_distribution, x, data, p0=guess_params,maxfev = 100000)
            condition_number = np.linalg.cond(cov)

            if (condition_number > 10 ** 18) & (i != end_image):
                print('Image number', i, 'appears to be unfittable')
                with Image.open(image_filename_prefix + filename_postfix(i-1 + frames_without_bleaching)) as image_pre:
                    image_pre = image_pre.convert('L')
                    data_pre = np.array(image_pre).astype(float) / 255
                data_pre = data_pre[crop_x:-crop_x, crop_y:-crop_y]
                data_pre = np.mean(data_pre, 1)
                data_pre /= baseline

                with Image.open(image_filename_prefix + filename_postfix(i+1 + frames_without_bleaching)) as image_post:
                    image_post = image_post.convert('L')
                    data_post = np.array(image_post).astype(float) / 255
                data_post = data_post[crop_x:-crop_x, crop_y:-crop_y]
                data_post = np.mean(data_post, 1)
                data_post /= baseline

                guess_params_pre = data_pre[0], min(data_pre) - data_pre[0], x[np.argmin(data_pre)], 0.2 * image_size_px, (min(data_pre) + data_pre[0])/50, x[np.argmin(data_pre[:round(0.5*len(x))])], 0.4 * image_size_px
                params_pre, cov_pre = curve_fit(double_gaussian_distribution, x, data_pre, p0=guess_params_pre, maxfev = 100000)

                guess_params_post = data_post[0], min(data_post) - data_post[0], x[np.argmin(data_post)], 0.2 * image_size_px, (min(data_post) + data_post[0])/50, x[np.argmin(data_post[:round(0.5*len(x))])], 0.4 * image_size_px
                params_post, cov_post = curve_fit(double_gaussian_distribution, x, data_post, p0=guess_params_post, maxfev = 100000)



                params_mean = [(g + h) / 2 for g, h in zip(params_pre, params_post)]
                params = params_mean
            if (condition_number > 10 ** 18) & ((i == 24) | (i == 25) | (i == 26)):
                print('The last image appears to be unfittable')
                with Image.open(image_filename_prefix + filename_postfix(i-1 + frames_without_bleaching)) as image_pre:
                    image_pre = image_pre.convert('L')
                    data_pre = np.array(image_pre).astype(float) / 255
                data_pre = data_pre[crop_x:-crop_x, crop_y:-crop_y]
                data_pre = np.mean(data_pre, 1)
                data_pre /= baseline

                guess_params_pre = data_pre[0], min(data_pre) - data_pre[0], x[np.argmin(data_pre)], 0.2 * image_size_px, (min(data_pre) + data_pre[0])/50, x[np.argmin(data_pre[:round(0.5*len(x))])], 0.4 * image_size_px
                params_pre, cov_pre = curve_fit(double_gaussian_distribution, x, data_pre, p0=guess_params_pre, maxfev = 100000)

                params = params_pre
            w.append(params[-4])
            w1.append(params[-1])

        else:
            guess_params = data[0], min(data) - data[0], x[np.argmin(data)], 0.2 * image_size_px
            params, cov = curve_fit(gaussian_distribution, x, data, p0=guess_params,maxfev = 100000)
            standard_deviation = np.sqrt(np.diag(cov))
            w.append(params[-1])

        #if i < show_curves + 2:
        if i%show_curves == 0:
            plot_color = next(plt.gca()._get_lines.prop_cycler)['color']
            plt.plot(x, data, linewidth=0.2, color=plot_color, label='_nolegend_')
            if curves_number == 1:
              plt.plot(x, gaussian_distribution(x, *params), color=plot_color)
            else:
              plt.plot(x, double_gaussian_distribution(x, *params), color=plot_color)

    if show_curves > 0:
        plt.xlabel('x, μm')
        plt.ylabel('Intensity')
        plt.show()
    if curves_number == 2:
        return w, w1
    else:
        return w

In [ ]:
def process_diffusion(w, t, w1=None, linear_points=None, remove_outliers=False, show_plot=False,
                      room_temperature_celsius=23., solvant_viscosity=0.9358e-3):
    """Calculates diffusion coefficient and hydrodynamic diameter of particles based on FRAP data.

    Parameters
    ----------
    w : iterable
        Array of widths at half minimum intensity of bleached line (main component)
    w1 : iterable
        Array of widths at half minimum intensity of bleached line (2nd component)
    t : float, iterable
        Either a time per frame or an array of time delays inbetween frames
    linear_points : int, optional
        Number of points from which the slope is determined. If None, all points are considered (None by default)
    remove_outliers : bool, iterable, optional
        List of indices of outliers to be removed from `w`
        If False, no points are removed (False by default)
        If True, outliers are removed automatically (not implemented yet)
    show_plot : bool, optional
        If True, shows plot of :math:`w^2(t)`, False by default
    room_temperature_celsius : float, optional
        Room temperature in Celsius, 23 by default
    solvant_viscosity : float, optional
        Solvant viscosity in Pa * s, 0.9358e-3 by default (water at 23 degrees Celsius)

    Returns
    ----------
    float : diffusion coefficient, m^2 / s
    float : diffusion coefficient 1, m^2 / s
    float : hydrodynamic radius, nm
    float : hydrodynamic radius 1, nm
    """

    if isinstance(t, float):
        t = np.arange(len(w)) * t

    w2 = np.array(w) ** 2
    if w1 != None:
        w12 = np.array(w1) ** 2

    if isinstance(remove_outliers, Iterable):
       w = np.delete(w, remove_outliers)
       t = np.delete(t, remove_outliers)
    elif remove_outliers:
       TODO: remove outliers automatically
       pass

    if linear_points is None:
        linear_points = len(w)

    k, b = np.polyfit(t[:linear_points - 1], w2[:linear_points - 1], 1)
    if w1 != None:
        k1, b1 = np.polyfit(t[:linear_points - 1], w12[:linear_points - 1], 1)

    if show_plot:
        plt.plot(t, w2, marker='o', linewidth=0)
        plt.plot(t, t * k + b)
        if w1 != None:
            plt.plot(t, w12, marker='o', linewidth=0)
            plt.plot(t, t * k1 + b1)
        plt.xlabel('Time, s')
        plt.ylabel('w², μm²')
        plt.grid()
        plt.show()

    room_temperature_kelvin = room_temperature_celsius + zero_Celsius
    diffusion_coeff = k * 1e-12 / 8  # m^2 / s
    diameter = k_b * room_temperature_kelvin / (3 * np.pi * solvant_viscosity * diffusion_coeff)
    if w1 != None:
        diffusion_coeff_1 = k1 * 1e-12 / 8  # m^2 / s
        diameter_1 = k_b * room_temperature_kelvin / (3 * np.pi * solvant_viscosity * diffusion_coeff_1)
    if w1 != None:
        return diffusion_coeff, diameter, diffusion_coeff_1, diameter_1
    else:
        return diffusion_coeff, diameter

In [ ]:
if __name__ == '__main__':
    w, w1 = process_frap_images(directory_path='/content/drive/MyDrive/frap/3',
                            curves_number = 2,
                            crop=(0.1, 0.01),
                            scan_size_um=850,
                            frames_without_bleaching=3,
                            show_baseline=True,
                            show_curves=5)
    D_1, diameter_1, D_2, diameter_2 = process_diffusion(w,
                                    t=3.,
                                    w1 = w1,
                                    solvant_viscosity = 6e-3,
                                    show_plot=True)
    print(f'Hydrodynamic diameter 1: {round(diameter_1 * 1e9, 5)} nm')
    print(f'Diffusion coefficient 1: {round(D_1, 20)} m^2/s')
    print(f'Hydrodynamic diameter 2: {round(diameter_2 * 1e9, 5)} nm')
    print(f'Diffusion coefficient 2: {round(D_2, 20)} m^2/s')
    #print(number_of_images)

In [ ]:
if __name__ == '__main__':
    w = process_frap_images(directory_path='/content/drive/MyDrive/frap/3',
                            curves_number = 1,
                            crop=(0.1, 0.01),
                            scan_size_um=850,
                            frames_without_bleaching=2,
                            show_baseline=False,
                            show_curves=5)
    D_1, diameter_1 = process_diffusion(w,
                                    t=3.0,
                                    solvant_viscosity = 6e-3,
                                    show_plot=True)
    print(f'Hydrodynamic radius: {round(diameter_1 * 1e9 / 2, 3)} nm')
    print(f'Diffusion coefficient: {round(D_1, 20)} m^2/s')
    print(number_of_images)